# Visualizing Models:

- In this Notebook we show how to set up a bare Meshcat visualizer, and add a bunch of objects. Later we will interact with Meshcat through the system.frameworks methods DiagramBuilder, SceneGraph, MultibodyPlant etc...

In [17]:
from pydrake.geometry import StartMeshcat
from IPython.display import IFrame

print("Click the link to open Meshcat viewer in the browser...")
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7001


Click the link to open Meshcat viewer in the browser...


## OR...
***You can do it INSIDE your notebook***: though this can get a little messy, because you'll be adding code and having to scroll up to see, but it can be handy for debugging.

In [18]:
address = meshcat.web_url()
IFrame(src=address, width='100%', height='500px')

# Add some objects

In [19]:
from pydrake.geometry import Box, Capsule, Ellipsoid
from pydrake.math import RigidTransform
from pydrake.geometry import Rgba

# width, depth, height
box = Box(2, 2, 2)
meshcat.SetObject("box0", box, Rgba(r=0.9, g=0.3, b=0.3, a=1.0))
meshcat.SetTransform("box0", RigidTransform([0, 0, 1]))

# Can re-use other box object to initialize shape
# and still freely transform as a unique Shape
meshcat.SetObject("box1", box, Rgba(r=0.9, g=0.7, b=0.3, a=1.0))
meshcat.SetTransform("box1", RigidTransform([4, 0, 1]))

# radius, height
capsule = Capsule(0.5, 2)
meshcat.SetObject("capsule0", capsule, Rgba(r=0.3, g=0.7, b=0.3, a=1.0))
meshcat.SetTransform("capsule0", RigidTransform([4, 4, 4]))

# a,b,c lengths of principal semi-axes
ellipsoid = Ellipsoid(0.5, 0.75, 2)
meshcat.SetObject("ellipsoid0", ellipsoid, Rgba(r=0.3, g=0.7, b=0.7, a=1.0))
meshcat.SetTransform("ellipsoid0", RigidTransform([-4, 4, 4]))

# Add Buttons and Sliders

- Verify in meshcat window that the buttons/sliders show up

In [20]:
# Labelled button
meshcat.AddButton("button_we_delete_later")

# Labelled button with keydown callback for Left Arrow key:
# Possible keycodes:
# https://developer.mozilla.org/en-US/docs/Web/API/UI_Events/Keyboard_event_code_values
meshcat.AddButton("button1", keycode="ArrowLeft")

meshcat.DeleteButton('button_we_delete_later')

# Return numclicks
meshcat.GetButtonClicks('button1')

0

- Verify in meshcat window that ArrowDown/ArrowUp and ArrowLeft can control slider1 / button1

In [21]:
# Labelled slider
meshcat.AddSlider("slider0", 0, 100, 1, 25)

# All options, explicitly:
meshcat.AddSlider("slider1", min=0, max=100, step=1, value=50, 
                  decrement_keycode="ArrowDown", 
                  increment_keycode="ArrowUp"
)

print(meshcat.GetSliderNames())
meshcat.DeleteSlider("slider0")
print(meshcat.GetSliderNames())
print(meshcat.GetSliderValue('slider1'))

meshcat.SetSliderValue('slider1', 79) 

['slider0', 'slider1']
['slider1']
50.0


***Add an environment map, to make it look like a disco***.

- Note: Check the docs, adding just any old image won't get you very good results. You can download free, or build them in Blender etc...

In [23]:
# Cool!
meshcat.SetEnvironmentMap('testenvironmentmap.jpg')

***PlotSurface()***

In [24]:
import numpy as np
from pydrake.geometry import Rgba

#PlotSurface(self: pydrake.geometry.Meshcat, 
#            path: str, 
#            X: numpy.ndarray[numpy.float64[m, n], flags.f_contiguous], 
#            Y: numpy.ndarray[numpy.float64[m, n], flags.f_contiguous], 
#            Z: numpy.ndarray[numpy.float64[m, n], flags.f_contiguous], 
#            rgba: pydrake.geometry.Rgba = Rgba(r=0.1, g=0.1, b=0.9, a=1.0), 
#            wireframe: bool = False, 
#            wireframe_line_width: 
#            float = 1.0) → None


# Plot the six-hump camel
xs = np.linspace(-4, 4, 51)
ys = np.linspace(-4, 4, 51)
[X, Y] = np.meshgrid(xs, ys)
P = 0.3*(X**2 + Y**2)

meshcat.PlotSurface(path="quadratic_bowl",
                    X=X,
                    Y=Y,
                    Z=P,
                    rgba=Rgba(0.8, 0.8, 0.3),
                    wireframe=False,
                    wireframe_line_width=2.0)

...wireframe...

In [25]:
meshcat.PlotSurface(path="quadratic_bowl_wireframe",
                    X=X,
                    Y=Y,
                    Z=P,
                    rgba=Rgba(0.5, 0.1, 0.8),
                    wireframe=True,
                    wireframe_line_width=2.0)

***SetLine()***

In [26]:
from pydrake.geometry import Rgba

# Must be 3xn (so transpose)
verts = np.array([[0,0,0],
                     [0,10,0],
                     [0,10,10],
                     [10,10,10],
                     [10,0,10],
                     [10,0,0],
                     [0,0,0]]).T

meshcat.SetLine("newThingaMajig", verts, line_width=2.0, rgba=Rgba(r=1.0, g=1.0, b=1.0, a=1.0))

***SetLineSegments()***

In [27]:
from pydrake.geometry import Rgba

# Must be 3xn (so transpose)
start_verts = np.array([[0,10,0],
                 [0,10,10],
                 [10,10,10],
                 [10,0,10],
                 [10,0,0]]).T

end_verts = np.array([[0,0,0],
                 [0,0,0],
                 [0,0,0],
                 [0,0,0],
                 [0,0,0]]).T

meshcat.SetLineSegments("newThingaMajig2", 
                        start_verts,
                        end_verts, 
                        line_width=2.0, 
                        rgba=Rgba(r=1.0, g=1.0, b=0.0, a=1.0))

***Move things around with SetTransform()***

In [29]:
from pydrake.math import RollPitchYaw, RotationMatrix

position = [4,0,1.5]
rpy = RollPitchYaw(np.array([[1, 2, 3]]).T)
pose34 = RotationMatrix(rpy)
X = RigidTransform(RotationMatrix(rpy), position)

meshcat.SetTransform("box1", X)

***SetTriangleColorMesh() and SetTriangleMesh()***

In [31]:
# List of verts in shape
verts = np.array([[0,0,0],
                  [2,0,0],
                  [1,3,0],
                  [0,0,2]
                 ]).T

# List of vert indices corresponding to face
faces = np.array([[0,1,2],
                  [0,1,3],
                  [0,2,3],
                  [2,3,1]
                 ]).T

# rgb corresponding to each vertex
colors = np.array([[1,0,0],
                   [0,1,0],
                   [0,0,1.0],
                   [1,1,0]
                  ]).T


########## SetTriangleColorMesh ##############
# Vertex colors, blend across face:
meshcat.SetTriangleColorMesh("handMesh", 
                             vertices=verts, 
                             faces=faces,
                             colors=colors,
                             wireframe=False, 
                             wireframe_line_width=1.0,
                             side=meshcat.SideOfFaceToRender.kDoubleSide)


########## SetTriangleMesh ##############
# One color for the whole object (with alpha)
meshcat.SetTriangleMesh("handMesh2", 
                             vertices=verts, 
                             faces=faces,
                             rgba=Rgba(r=0.9, g=0.3, b=0.3, a=0.5),
                             wireframe=False, 
                             wireframe_line_width=1.0,
                             side=meshcat.SideOfFaceToRender.kDoubleSide)



position = [-8,1,2]
rpy = RollPitchYaw(np.array([[1, 2, 3]]).T)
X2 = RigidTransform(RotationMatrix(rpy), position)
meshcat.SetTransform("handMesh", X2)

position = [6,-4,2]
X3 = RigidTransform(RotationMatrix(rpy), position)
meshcat.SetTransform("handMesh2", X3)

***If you need more control, use MeshCatParams***

- Note: StartMeshCat is just initializing some extra Deepnote friendly options, but internally it calls Meshcat(), as below:

In [ ]:
from pydrake.geometry import MeshcatParams

meshcat_params = MeshcatParams(
    host='http://localhost',
    port=7000,                               # must be zero or >= 1024
    show_stats_plot=True#,                   # plot including realtime rate and WebGL render statistics.
    web_url_pattern='http://{host}:{port}'   # This may be useful in case Meshcat sits behind a firewall or proxy.
)

# and call
meshcat = Meshcat(meshcat_params)